### Import required libraries

In [3]:
import pickle
import pandas as pd
import numpy as np
import os

from pathlib import Path
from scipy.io import loadmat
from sklearn.decomposition import PCA

from torch.utils.data import Dataset, DataLoader, SequentialSampler
import torchvision.transforms as transforms
from PIL import Image
import glob 
import torch

from IPython.display import display
from copydf import copyDF

### Define functions to compute principal components 

In [4]:
from scipy.stats import pearsonr

def get_principal_components(all_feats, n_components, verbose=False):
    pca = PCA(n_components=n_components, svd_solver='full')
    feats = pca.fit_transform(all_feats)
    if verbose:
        print(pca.explained_variance_ratio_)
        print(np.cumsum(pca.explained_variance_ratio_))
    return feats, len(pca.explained_variance_ratio_)

def get_coefficients(feats, df, n_components, verbose=False):
    pc, components = get_principal_components(feats, n_components, verbose=verbose)
    
    components_to_return = components
    print(f"{components} components required")
    components = min(components, 30)
    coefficients = {}
    principles = ['Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form', 'multiplicity-vs-unity',
       'absolute-clarity-vs-relative-clarity']
    
    for p in principles:
        for c in range(components):
            coefficients.setdefault(p,[]).append(pearsonr(df[p].values, pc[:, c])[0])
        
    return pd.DataFrame(coefficients), components_to_return

### Find best correlating wofflin's principles for real art

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [11]:
# get indexes from all datasets to take subset of the features for which we have wolflins principles.
df = pd.read_csv('datasets/processed/combined_wofflins_real.csv')
df['name'] = df['Input.image'].apply(lambda x: Path(x).name)
mat_file = loadmat('datasets/processed/groundtruth_pruned.mat')
files = [Path(f[0][0]).name for f in mat_file['groundtruth_pruned'][0][0][0]]
real_indexes = [files.index(f) for f in df.name.values]
df.shape

(1000, 7)

In [15]:
folder = 'datasets/features/real_art_features'
table_for_n_components = {}
dataframes = {}
for file in os.listdir(folder):
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)
        
    coefficient_df, c = get_coefficients(feats[real_indexes], df, 0.95)
    table_for_n_components[file] = c
    dataframes[file] = coefficient_df.transpose()
    print(file)
    display(coefficient_df.transpose())
    print('##'*80)   
    

25 components required
resnet101_plus2_real.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Linearly-vs-Painterly,0.051553,0.053265,-0.050186,-0.013625,-0.199060,0.084302,-0.069139,0.116331,-0.022185,-0.066643,0.070077,-0.008649,-0.014978,0.005222,0.086376,-0.025464,0.055999,-0.044623,-0.013140,-0.033152,-0.000766,-0.082861,-0.040415,0.001798,-0.027783
Planar-vs-Recessional,0.005949,0.030084,0.029188,0.008754,0.003897,0.054514,0.001552,0.015699,-0.053215,0.007784,0.011422,0.057624,-0.014396,0.083528,0.031117,0.092611,-0.148408,0.012588,-0.052206,0.002925,-0.022871,-0.081842,-0.020239,0.039938,0.096174
closed-form-vs-open-form,0.092326,0.001361,0.018076,-0.040970,-0.051898,-0.005046,-0.063008,-0.040866,-0.011608,-0.095841,0.056480,0.000166,-0.025959,0.102303,-0.021663,0.128942,-0.109370,-0.019607,-0.074493,0.013713,0.056529,-0.042398,-0.034927,0.078939,0.087548
multiplicity-vs-unity,0.033990,0.015185,-0.036012,-0.018280,-0.065400,0.093611,-0.037331,0.105423,-0.044458,-0.039646,0.065064,0.024988,-0.068778,-0.009979,0.044278,0.091129,-0.147571,0.009729,-0.011733,0.020500,0.046667,-0.090148,-0.061461,0.032845,0.031698
absolute-clarity-vs-relative-clarity,0.061150,0.028698,-0.077909,-0.083286,0.049035,-0.052848,-0.097174,-0.057076,-0.094344,-0.124909,0.070590,0.012980,-0.077804,0.097153,0.031308,0.091898,-0.256189,-0.056564,0.037186,0.089471,0.043354,-0.073160,-0.039002,0.020082,0.081847


################################################################################################################################################################
252 components required
resnet101_pretrained_real.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.248015,-0.080630,0.110035,-0.214783,0.291975,-0.033868,0.286823,0.009169,-0.017633,0.092347,-0.112583,0.070425,-0.001384,-0.009849,-0.073823,0.038563,0.076361,0.041687,-0.052902,-0.145701,-0.060188,-0.014902,0.048666,-0.076167,0.071780,0.122149,0.074552,0.000366,0.000606,-0.095856
Planar-vs-Recessional,-0.172845,0.264584,0.001578,-0.249369,0.080508,-0.118267,-0.017303,0.255669,0.084368,0.081440,0.054709,-0.017693,-0.106589,0.074340,-0.074211,0.001920,0.028889,-0.010946,-0.038707,-0.048736,0.095261,-0.008624,0.053594,-0.058105,-0.037138,0.054493,0.091968,0.069999,0.010347,0.018887
closed-form-vs-open-form,-0.334529,0.313968,0.091376,-0.090689,0.246443,-0.023781,-0.000659,0.141071,0.111506,0.082117,-0.082243,-0.163035,-0.178249,0.109700,0.008332,0.010244,0.059540,0.057056,-0.076188,-0.011602,0.062041,0.027169,0.008358,-0.046616,-0.090560,-0.025605,0.154549,0.017406,0.090883,0.074296
multiplicity-vs-unity,-0.227116,0.349667,0.014832,-0.285148,0.248886,-0.032405,0.042245,0.025313,0.047252,0.149264,-0.063401,0.090014,-0.134044,0.027127,-0.086611,0.073461,0.104290,0.052751,-0.099407,-0.023526,0.013497,0.022325,0.050063,-0.040951,-0.025187,0.076031,0.064597,0.100029,0.041504,-0.043214
absolute-clarity-vs-relative-clarity,-0.098177,0.543336,-0.325585,-0.087766,0.233906,0.137954,-0.043485,0.063782,0.041531,0.058957,-0.069154,-0.033758,-0.171322,0.144861,-0.065483,0.120848,0.071229,0.050088,-0.122084,-0.016274,0.043939,0.060321,0.022342,-0.024031,-0.012955,-0.011006,0.028761,0.044792,0.098291,0.012376


################################################################################################################################################################
26 components required
resnet50_plus2_real.pkl


...

################################################################################################################################################################
362 components required
wikiart_disc_sg2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.356158,-0.096498,0.084278,-0.119796,0.165128,0.058816,-0.070095,0.157564,-0.083772,0.008850,0.016727,0.022862,-0.034966,-0.066218,-0.066603,0.035426,0.035140,-0.075129,0.091667,-0.022405,0.001737,0.223848,-0.077206,0.019544,0.089652,-0.003538,-0.003023,-0.089675,-0.020572,0.049475
Planar-vs-Recessional,-0.054054,-0.151556,-0.256969,-0.012397,0.149099,0.102475,-0.071387,0.221811,-0.154345,0.055488,0.046996,0.019156,0.044519,-0.086373,-0.030999,-0.071769,-0.016172,-0.149023,0.024262,0.124262,-0.023452,0.044997,0.090660,-0.074281,0.006308,0.030299,0.054052,-0.033261,-0.099386,0.039051
closed-form-vs-open-form,-0.215841,-0.168172,-0.368549,-0.006542,0.087325,0.321021,0.039673,0.265574,-0.110660,0.178268,-0.052204,-0.012799,-0.090989,-0.104524,-0.020186,0.026808,-0.091682,-0.071248,-0.038222,0.010444,-0.001175,-0.021358,0.087690,-0.017041,0.025657,0.013094,0.022904,-0.027183,-0.018474,0.034123
multiplicity-vs-unity,-0.226588,-0.174046,-0.283005,-0.037130,0.307494,0.138025,0.047540,0.210653,-0.105061,-0.034187,0.047549,0.072844,-0.026600,-0.090820,-0.158860,-0.034069,0.055525,-0.064826,0.000108,0.000742,-0.056009,0.126252,0.049484,0.080367,0.014972,0.007837,0.089211,-0.061006,-0.080267,0.003583
absolute-clarity-vs-relative-clarity,-0.069011,0.041455,-0.687491,0.094165,0.199758,0.044045,0.176186,0.169369,-0.120569,-0.016995,0.011843,0.000625,-0.016483,-0.005414,-0.077730,0.005952,0.071654,0.007992,-0.025162,-0.006963,-0.018273,0.024025,0.078241,0.078402,0.029765,-0.010367,0.038132,-0.031669,-0.025271,-0.011727


################################################################################################################################################################
652 components required
wikiart_disc_sg2_can.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,0.095049,-0.011161,-0.270450,0.138466,-0.095470,-0.071981,-0.252751,-0.092542,0.026228,0.078910,0.046384,0.019458,-0.100688,0.127001,0.055624,0.063342,0.071840,0.085121,-0.041940,-0.051272,0.084956,0.034599,-0.119708,0.077620,0.002112,-0.067877,-0.013183,0.123140,-0.025997,0.052959
Planar-vs-Recessional,0.085841,0.193270,0.010459,0.345380,0.042821,-0.082675,-0.136553,-0.005047,0.120454,0.043083,0.163376,0.082998,0.092326,-0.035842,0.053526,-0.040542,0.002390,0.037622,-0.007414,-0.003051,0.084142,0.086099,-0.063935,0.012254,0.010262,-0.054177,0.017896,-0.010119,0.026507,-0.022592
closed-form-vs-open-form,0.189472,0.426265,-0.126859,0.282943,0.031447,0.126787,-0.108922,-0.117415,0.109421,-0.026248,0.047512,0.074532,0.124144,-0.024615,0.052157,-0.033525,0.039526,0.099672,0.066957,0.041624,0.128758,0.120414,-0.060875,0.012795,-0.017632,-0.095628,0.070538,-0.065345,-0.017748,-0.024923
multiplicity-vs-unity,0.099358,0.320844,-0.013680,0.351742,-0.003932,-0.088130,-0.284505,0.031470,0.016559,0.078908,0.063723,0.010387,-0.035142,0.138138,0.026203,0.035574,0.015272,0.096227,0.006895,-0.024598,0.055329,0.083312,-0.001361,0.037739,-0.090131,-0.072122,0.075111,0.051193,-0.038107,0.001873
absolute-clarity-vs-relative-clarity,-0.209267,0.563981,0.235535,0.311125,-0.003662,0.128525,-0.191312,-0.011020,0.030805,-0.031807,0.016468,0.064615,0.058780,0.032536,0.073209,0.012767,-0.068178,0.067027,0.025540,0.012647,0.113140,0.037653,-0.002923,0.001478,-0.043208,-0.027517,0.034652,-0.021258,-0.035109,-0.031228


################################################################################################################################################################


In [12]:
# copy to clipboard.
copyDF( dataframes['resnet101_plus2_real.pkl'] )

<IPython.core.display.Javascript object>

In [20]:
table_for_n_components = {k: [v] for k,v in table_for_n_components.items()}

In [21]:
pd.DataFrame(table_for_n_components)

,resnet101_plus2_real.pkl,resnet101_pretrained_real.pkl,resnet50_plus2_real.pkl,resnet50_pretrained_real.pkl,vgg16_plus2_real.pkl,vgg16_pretrained_real.pkl,wikiart_disc_sg1.pkl,wikiart_disc_sg1_can.pkl,wikiart_disc_sg2.pkl,wikiart_disc_sg2_can.pkl
0,25,252,26,262,14,548,125,122,362,652


### StyleGAN2

In [12]:
generations_csv = {
'StyleGAN2' : ('datasets/features/generated_artworks/StyleGAN2/', 'datasets/processed/combined_wofflins_styleGAN2.csv'),
'StyleCAN2' : ('datasets/features/generated_artworks/StyleCAN2/', 'datasets/processed/combined_wofflins_styleCAN2.csv')
}

generated_indexes = {}
generated_csvs = {}

In [13]:
for k, (g, c) in generations_csv.items():
    items = glob.glob(g + '*')
    items = [i for i in items if 'lowest_shape_entropy' not in i]
    csv = pd.read_csv(c)
    names = list(csv['Input.image'].apply(lambda x: Path(x).name).values)
    indexes = [names.index(Path(i).name) for i in items]
    generated_csvs[k] = csv
    generated_indexes[k] = indexes

In [15]:
folder = 'datasets/features/generated_art_features/'
table_for_n_components = {}
dataframes = {}
for file in os.listdir(folder):
    generation_name = file.split('_')[-1].split('.')[0]
    if generation_name not in ['StyleGAN2']:
        continue
    
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)

    generated_csvs[generation_name].columns = ['Input.image', 'multiplicity-vs-unity', 'absolute-clarity-vs-relative-clarity', 'Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form']
    coefficient_df, c = get_coefficients(feats, generated_csvs[generation_name], 0.95)
    table_for_n_components[file] = c
    dataframes[file] = coefficient_df.transpose()
    print(file)
    display(coefficient_df.transpose())
    print('##'*80) 

91 components required
StyleGAN1_StyleGAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.302282,-0.128386,-0.025853,-0.012486,-0.083155,-0.135066,0.045026,-0.248375,-0.038277,-0.005134,0.028290,0.098607,-0.142397,0.038165,0.106068,0.053886,-0.001289,-0.048989,-0.072776,-0.053701,0.001492,-0.088446,-0.112217,0.099234,-0.000991,-0.073658,0.029873,-0.027696,0.048033,0.013363
Planar-vs-Recessional,-0.146923,-0.114356,-0.183533,-0.057985,-0.141574,-0.199106,-0.093536,-0.080410,-0.023426,0.044026,0.013796,0.115950,-0.079684,0.037095,0.047391,0.039011,0.044503,0.027623,0.016355,0.042029,-0.097330,-0.031439,-0.067140,0.078658,-0.159591,0.029250,-0.089431,-0.031976,-0.078054,-0.056143
closed-form-vs-open-form,-0.240167,-0.106199,-0.199292,0.051082,0.042304,-0.130778,-0.002891,-0.018668,-0.081163,0.081713,0.035503,0.156563,-0.115035,-0.071724,0.141696,0.050324,0.087538,0.101574,-0.020726,-0.016835,-0.033510,0.040197,-0.036954,0.093053,-0.061579,0.036012,-0.066125,-0.123479,-0.036109,0.010460
multiplicity-vs-unity,0.071576,-0.113385,-0.000475,0.033368,-0.093652,0.010095,0.107949,-0.137473,0.073492,-0.004260,-0.016671,-0.065276,-0.087563,-0.056439,-0.079822,0.061925,0.107574,-0.030393,-0.066048,0.099262,-0.011691,-0.019991,0.071642,-0.004809,-0.034087,-0.019867,-0.050850,-0.033587,-0.043396,0.010201
absolute-clarity-vs-relative-clarity,-0.205472,-0.072367,-0.276976,0.081416,-0.069520,0.093413,0.061707,-0.023159,0.007298,-0.034040,0.092312,0.020461,-0.150058,0.022978,-0.157103,0.008385,0.133884,-0.057221,0.004708,0.020379,-0.054327,0.089491,0.055783,-0.015304,-0.113535,-0.063289,0.046058,-0.072943,-0.092148,0.047367


################################################################################################################################################################
77 components required
StyleCAN1_StyleGAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.393789,0.025940,-0.167872,0.037060,0.103318,0.018732,0.065068,-0.196371,0.049748,0.030675,0.121905,-0.025349,0.044340,-0.068958,0.030477,-0.012417,-0.176043,-0.050316,0.007605,-0.023970,-0.003537,0.108456,0.027496,-0.009211,0.010981,0.024423,-0.045612,-0.077250,-0.041274,-0.018347
Planar-vs-Recessional,-0.331127,0.039436,-0.180921,0.116355,-0.147980,0.057564,-0.010152,-0.091280,-0.111710,-0.030950,0.093902,0.048492,-0.110233,-0.009302,0.009683,-0.086993,-0.060725,0.086266,-0.035454,-0.009466,-0.142725,0.083305,-0.056345,0.020627,-0.098372,0.109697,-0.069553,-0.001383,-0.080177,-0.058719
closed-form-vs-open-form,-0.238845,0.162502,-0.119367,-0.048040,0.065637,-0.021080,-0.024123,-0.107686,0.019626,-0.043796,0.197829,0.029412,0.030170,0.113809,0.014143,-0.077952,-0.053736,-0.045393,0.071753,0.072654,-0.193929,-0.000267,-0.061173,0.047025,-0.060536,0.040958,-0.097016,0.060272,-0.000918,-0.062177
multiplicity-vs-unity,-0.065480,-0.058414,-0.016943,0.038836,-0.070258,0.113169,-0.072516,-0.309196,-0.031629,-0.068822,-0.035007,0.029803,0.046161,-0.040554,0.007256,-0.130450,-0.049243,0.062588,-0.030100,0.057313,-0.048542,0.010295,-0.044240,0.049582,0.010459,-0.032981,-0.010649,0.011240,0.072680,-0.022610
absolute-clarity-vs-relative-clarity,-0.193751,0.281732,-0.152182,0.092608,-0.089400,-0.051401,-0.171436,-0.317224,-0.118383,-0.061448,-0.075581,0.005387,0.025322,0.031291,0.042211,-0.082455,-0.038819,-0.104717,-0.110104,0.003857,-0.048250,-0.010138,-0.072120,0.040053,-0.005919,0.085305,0.029947,-0.003767,-0.025272,0.021717


################################################################################################################################################################
190 components required
StyleGAN2_StyleGAN2.pkl


...

################################################################################################################################################################
329 components required
vgg16_StyleGAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,0.045119,0.376047,0.042093,0.053177,-0.184499,-0.120832,-0.015994,0.069356,0.053509,-0.030962,-0.061083,0.115066,0.027806,0.003385,-0.002355,-0.035017,0.048345,0.002409,-0.088090,0.044040,0.066569,0.025809,-0.108811,0.015843,-0.006676,0.007633,0.111341,0.060951,0.010620,0.021482
Planar-vs-Recessional,0.043604,0.211411,0.157134,-0.150405,-0.019750,-0.146578,-0.071585,0.068691,0.034634,-0.018655,-0.050398,0.134741,0.030263,-0.034321,-0.034226,0.025048,-0.058564,-0.090171,-0.025280,0.069054,0.062896,0.030087,-0.039968,0.012879,-0.049005,-0.006516,0.088096,0.110765,-0.017695,-0.008684
closed-form-vs-open-form,0.016510,0.267812,-0.034464,-0.099441,-0.072728,-0.058598,-0.028419,0.042140,0.118170,-0.055801,0.024559,0.063591,0.083272,0.045501,-0.043382,-0.002874,-0.017025,-0.078857,-0.089349,-0.008728,0.098073,0.065707,-0.039284,0.019365,0.000202,-0.088395,-0.060142,0.012150,0.054210,0.033555
multiplicity-vs-unity,0.119408,-0.080213,-0.018881,-0.083990,-0.055884,-0.111647,0.006732,0.068053,-0.049349,0.023892,-0.107750,-0.034678,0.168294,0.019593,0.021477,-0.001970,0.081086,-0.073589,-0.041562,0.051530,0.100305,-0.044520,-0.012445,-0.118987,0.015648,-0.044893,0.057012,0.013161,-0.075432,0.004067
absolute-clarity-vs-relative-clarity,0.180830,0.224515,-0.039434,-0.146896,-0.038769,0.024662,-0.055016,0.203726,0.095913,-0.000780,0.007851,-0.032518,0.078622,-0.019145,0.120323,0.094048,-0.068303,0.012878,0.064697,0.071329,0.117934,0.008290,0.061731,-0.113805,0.061393,-0.024080,0.002655,0.049821,-0.130132,0.000713


################################################################################################################################################################
15 components required
vgg16_plus2_StyleGAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Linearly-vs-Painterly,0.099200,-0.071031,-0.161078,-0.180632,-0.077666,0.071603,0.168146,-0.061398,0.093637,0.039574,-0.000712,0.003747,0.044967,-0.018770,0.042497
Planar-vs-Recessional,-0.021112,0.014988,-0.108347,-0.119409,-0.169035,0.132195,0.110445,-0.103334,0.131998,-0.013209,-0.100167,-0.038531,0.007895,-0.077354,0.017792
closed-form-vs-open-form,-0.010675,-0.023331,-0.078058,-0.156410,-0.016630,0.027935,0.116175,-0.143925,-0.046076,0.056073,-0.088707,-0.029339,0.049295,0.069110,0.032352
multiplicity-vs-unity,-0.002187,-0.025581,-0.115938,0.070585,-0.063359,0.065542,-0.039647,0.001848,0.010512,0.027004,0.000975,0.091682,-0.048278,0.005543,0.004970
absolute-clarity-vs-relative-clarity,0.093591,-0.121955,-0.056815,-0.073493,-0.038830,0.026236,0.039027,0.003918,-0.037052,0.101317,0.083194,0.001281,-0.012663,-0.010280,-0.001445


################################################################################################################################################################


In [48]:
table_for_n_components = {k: [v] for k,v in table_for_n_components.items()}
pd.DataFrame(table_for_n_components)

,StyleGAN1_StyleGAN2.pkl,StyleCAN1_StyleGAN2.pkl,StyleGAN2_StyleGAN2.pkl,StyleCAN2_StyleGAN2.pkl,resnet50_StyleGAN2.pkl,resnet50_plus2_StyleGAN2.pkl,resnet101_StyleGAN2.pkl,resnet101_plus2_StyleGAN2.pkl,vgg16_StyleGAN2.pkl,vgg16_plus2_StyleGAN2.pkl
0,91,77,190,256,165,25,160,24,329,15


### StyleCAN2 

In [16]:
folder = 'datasets/features/generated_art_features/'
table_for_n_components = {}
dataframes = {}
for file in os.listdir(folder):
    generation_name = file.split('_')[-1].split('.')[0]
    if generation_name not in ['StyleCAN2']:
        continue
    
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)

    generated_csvs[generation_name].columns = ['Input.image', 'multiplicity-vs-unity', 'absolute-clarity-vs-relative-clarity', 'Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form']
    coefficient_df, c = get_coefficients(feats, generated_csvs[generation_name], 0.95)
    table_for_n_components[file] = c
    dataframes[file] = coefficient_df.transpose()
    print(file)
    display(coefficient_df.transpose())
    print('##'*80) 

94 components required
StyleGAN1_StyleCAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.336881,-0.096894,0.104821,-0.245529,-0.034890,-0.181474,-0.113072,0.088904,-0.033800,-0.099097,-0.038028,0.012071,0.081224,0.071495,0.132415,0.015123,-0.010439,0.091834,0.021656,-0.079176,0.001199,-0.201874,-0.051323,0.060621,0.062527,0.034813,0.020086,0.048052,0.060886,-0.030807
Planar-vs-Recessional,-0.113174,-0.106960,-0.097994,-0.172722,-0.002933,-0.030736,-0.094707,0.014922,-0.028452,-0.137539,-0.103327,-0.017996,0.012468,-0.026698,0.002524,0.119798,-0.040173,0.021568,0.029940,-0.014521,-0.051739,-0.078632,-0.071485,0.058359,0.050648,-0.093492,-0.035759,0.022998,0.001186,-0.030200
closed-form-vs-open-form,-0.219607,-0.060160,-0.164699,-0.173871,0.058506,-0.030487,-0.029683,-0.082898,0.011928,0.028717,0.108400,0.060687,-0.007673,0.054200,0.022530,0.173425,0.033580,-0.004327,0.062321,-0.074013,0.012103,-0.086008,-0.018909,-0.036906,0.059801,-0.015696,-0.037449,0.030300,-0.012435,-0.019700
multiplicity-vs-unity,-0.097495,-0.121421,-0.401105,-0.084472,-0.096114,-0.016183,-0.051137,0.121568,0.145115,0.071810,0.043785,0.005632,0.128493,-0.063662,0.067800,0.112129,-0.019224,-0.101220,0.001814,0.055263,0.046730,-0.155894,-0.031969,-0.011660,0.006902,-0.014433,-0.110338,-0.086742,-0.004790,-0.053522
absolute-clarity-vs-relative-clarity,-0.031921,-0.009880,-0.006907,-0.132492,0.079261,-0.084002,-0.078798,0.102492,-0.003101,0.033365,-0.111492,-0.048022,0.099008,0.016499,0.121593,0.098919,-0.062691,-0.050533,0.012812,-0.077671,-0.055772,-0.109503,0.046059,-0.004571,0.017366,0.012861,-0.092432,0.077663,-0.045861,-0.135090


################################################################################################################################################################
80 components required
StyleCAN1_StyleCAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.472980,-0.102451,-0.200150,-0.000253,-0.093421,0.029893,0.002499,0.072197,-0.036240,0.009617,-0.147900,0.016288,-0.117070,-0.149400,0.063962,-0.114607,0.058986,0.048394,0.107972,0.088111,0.031304,0.000821,0.021589,-0.060529,-0.053402,-0.118818,-0.001682,-0.057295,-0.026148,0.001456
Planar-vs-Recessional,-0.235240,0.078916,-0.064087,0.018844,-0.006954,0.007398,-0.096316,0.156429,0.047456,0.095630,-0.048860,0.024064,-0.045433,-0.033029,0.014209,-0.006745,0.040709,0.045709,0.079043,0.070177,-0.069765,-0.104189,-0.017164,-0.042131,0.039914,-0.083101,0.107040,-0.070501,-0.007989,0.035649
closed-form-vs-open-form,-0.212220,0.131074,-0.081698,-0.059245,0.140413,0.117481,-0.041407,0.067303,-0.058726,0.086541,0.015910,-0.048365,-0.051719,-0.074319,-0.072931,0.022593,0.046421,0.055980,0.099364,0.113158,-0.021828,-0.090655,0.020271,-0.032277,-0.034035,-0.073525,-0.011227,0.074138,-0.010945,0.025673
multiplicity-vs-unity,-0.104277,0.379117,-0.210027,0.055190,0.129942,0.012249,-0.290413,0.154758,-0.028249,0.054099,-0.033550,-0.029027,-0.075072,-0.026200,0.064953,-0.104571,0.120949,0.046918,0.047011,0.032663,-0.009716,-0.003611,0.022063,-0.025799,-0.046373,-0.063820,0.025342,0.039572,-0.044485,-0.100194
absolute-clarity-vs-relative-clarity,-0.146314,-0.012828,-0.024770,0.110006,-0.022502,0.088124,-0.178020,0.189936,-0.090339,-0.048505,-0.079616,-0.032034,-0.017736,-0.034721,0.042792,-0.101419,0.049623,-0.031886,0.109018,0.081985,0.015940,-0.007106,-0.022319,-0.059385,-0.003688,0.028543,-0.057870,0.004406,0.014393,0.005491


################################################################################################################################################################
199 components required
StyleGAN2_StyleCAN2.pkl


...

################################################################################################################################################################
330 components required
vgg16_StyleCAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Linearly-vs-Painterly,-0.063265,-0.439601,0.006741,-0.000867,-0.078717,0.117893,-0.025871,-0.136700,0.082655,0.129028,0.015278,-0.057116,-0.007594,-0.143531,0.118539,-0.083625,-0.051100,-0.084586,0.038456,0.025971,-0.045298,-0.003817,0.021558,-0.066483,0.057338,0.004281,-0.035784,-0.073667,-0.054220,-0.001300
Planar-vs-Recessional,-0.043939,-0.207542,0.053281,0.118584,0.022896,-0.016005,-0.032970,-0.164333,0.060776,0.018003,-0.000731,-0.057191,-0.012334,-0.008175,0.071858,0.056581,-0.110093,0.046977,-0.028280,-0.008380,-0.045119,0.015608,-0.106760,-0.033747,0.057833,-0.070572,0.000843,-0.009313,-0.004968,0.040135
closed-form-vs-open-form,-0.071156,-0.191738,0.138198,-0.044833,0.106566,-0.059780,0.034220,-0.032795,0.004939,0.072050,0.035146,0.061969,0.063329,0.034659,-0.040295,-0.040171,0.010136,0.060816,-0.054460,-0.025540,-0.080575,0.080304,-0.049951,0.024024,0.052666,0.006639,-0.027476,-0.015609,-0.030246,0.094096
multiplicity-vs-unity,0.106038,-0.108863,0.265345,0.175314,-0.130738,-0.027044,0.129686,-0.213127,-0.123890,0.069679,-0.010486,0.013543,0.051464,-0.024112,-0.060746,0.068224,-0.031475,0.092790,-0.037285,0.002631,-0.015959,0.004361,-0.048066,-0.046047,-0.085168,-0.000569,-0.061766,-0.000641,-0.030922,0.042259
absolute-clarity-vs-relative-clarity,0.062344,-0.128321,-0.002656,-0.001607,-0.104025,0.010575,0.024980,-0.027430,0.041205,0.016163,0.066494,-0.001833,0.005286,-0.022053,-0.022844,0.016681,-0.040480,0.047295,-0.071467,0.022858,-0.038170,0.026149,-0.091328,-0.034812,0.055237,-0.036602,-0.030098,-0.000887,0.008468,-0.021168


################################################################################################################################################################
16 components required
vgg16_plus2_StyleCAN2.pkl


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Linearly-vs-Painterly,-0.058977,-0.113960,-0.080948,0.172464,0.249743,0.069706,-0.133972,0.061623,0.087579,0.069111,0.082950,-0.095446,0.025898,-0.073364,-0.027409,-0.102870
Planar-vs-Recessional,-0.012121,-0.055665,-0.026470,0.073629,0.181571,0.105131,-0.060943,0.065639,-0.025293,-0.015584,0.029039,0.021058,-0.055358,0.033522,0.021073,0.032128
closed-form-vs-open-form,0.007987,-0.109212,-0.012951,0.041070,0.080443,0.052194,-0.026614,0.014934,-0.051569,-0.016068,0.053424,0.021957,0.017114,-0.018421,0.086136,0.010892
multiplicity-vs-unity,0.140104,-0.161870,-0.063835,-0.091042,0.022179,0.006898,-0.078624,0.020202,0.090031,-0.020490,-0.023331,0.053419,0.013269,-0.009793,0.033702,0.065249
absolute-clarity-vs-relative-clarity,-0.001062,-0.084564,-0.061928,-0.090660,0.129964,0.087381,-0.050737,-0.007216,0.003828,-0.027964,-0.030498,0.101372,-0.054556,-0.028855,0.063868,-0.041774


################################################################################################################################################################


In [46]:
table_for_n_components = {k: [v] for k,v in table_for_n_components.items()}
pd.DataFrame(table_for_n_components)

,StyleGAN1_StyleCAN2.pkl,StyleCAN1_StyleCAN2.pkl,StyleGAN2_StyleCAN2.pkl,StyleCAN2_StyleCAN2.pkl,resnet50_StyleCAN2.pkl,resnet50_plus2_StyleCAN2.pkl,resnet101_StyleCAN2.pkl,resnet101_plus2_StyleCAN2.pkl,vgg16_StyleCAN2.pkl,vgg16_plus2_StyleCAN2.pkl
0,94,80,199,272,169,26,161,25,330,16
